# Project Notebook

## General Imports

In [ ]:
import os
import json
import traceback

## Import the fablib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

## MFLib Imports

In [ ]:
import mflib 
print(f"MFLib version  {mflib.__version__} " )

from mflib.mflib import MFLib

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                    
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Establish the sets of nodes

"node_conf" will hold the collection of L3 Nodes
"net_conf" will hold the L2 nodes connecting the L3 Nodes
"route_conf" is the live connections between nodes

In [ ]:
#Needs polish
slice_name="topology-test-" + fablib.get_bastion_username()

[site1,site2,site3,site4,site5] = fablib.get_random_sites(count=5, avoid=["TOKY","FIU","CERN","DALL","GPN","LBNL","RENC","SALT","TACC","UKY","WASH","NCSA","LOSA","GATECH","INDI","MAX", "MASS","NEWY","SRI","UCSD"])

node_conf = [
 {'name': "consumer1", 'site': site1, 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']}, 
 {'name': "consumer2", 'site': site2, 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']}, 
 {'name': "server",  'site': site3, 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']}, 
 {'name': "router1", 'site': site4, 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}, 
 {'name': "router2", 'site': site5, 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}
]

net_conf = [
 {"name": "net_c1r1", "subnet": "10.10.1.0/24", "nodes": [{"name": "consumer1", "addr": "10.10.1.100"}, {"name": "router1", "addr": "10.10.1.10"}]},
 {"name": "net_c1r2", "subnet": "10.10.2.0/24", "nodes": [{"name": "consumer1", "addr": "10.10.2.200"}, {"name": "router2", "addr": "10.10.2.20"}]},
 {"name": "net_c2r2", "subnet": "10.10.3.0/24", "nodes": [{"name": "consumer2", "addr": "10.10.3.200"}, {"name": "router2", "addr": "10.10.3.20"}]},
 {"name": "net_sr1",  "subnet": "10.10.4.0/24", "nodes": [{"name": "server", "addr": "10.10.4.100"}, {"name": "router1", "addr": "10.10.4.10"}]},
 {"name": "net_sr2",  "subnet": "10.10.5.0/24", "nodes": [{"name": "server", "addr": "10.10.5.200"}, {"name": "router2", "addr": "10.10.5.20"}]}
]

route_config = [
    #consumer 1 links
    {"addr": "10.10.4.0/24", "gw": "10.10.1.10", "nodes": ["consumer1"]}, #can reach 10.10.4.0 (server) subnet through router1 gw
    {"addr": "10.10.3.0/24", "gw": "10.10.2.20", "nodes": ["consumer1"]}, #can reach 10.10.3.0 (consumer2) subnet through router2 gw
    {"addr": "10.10.5.0/24", "gw": "10.10.2.20", "nodes": ["consumer1"]}, #can reach 10.10.5.0 (server) subnet through router2 gw
    #consumer 2 (no fwd-ing link to rotuer 1 set-up here)
    {"addr": "10.10.2.0/24", "gw": "10.10.3.20", "nodes": ["consumer2"]}, #can reach 10.10.2.0 (consumer1) subnet through router2 gw
    {"addr": "10.10.5.0/24", "gw": "10.10.3.20", "nodes": ["consumer2"]}, #can reach 10.10.5.0 (server) subnet through router2 gw
    #server links
    {"addr": "10.10.1.0/24", "gw": "10.10.4.10", "nodes": ["server"]}, #can reach 10.10.1.0 (consumer1) subnet through router1 gw
    {"addr": "10.10.2.0/24", "gw": "10.10.5.20", "nodes": ["server"]}, #can reach 10.10.2.0 (consumer1) subnet through router2 gw
    {"addr": "10.10.3.0/24", "gw": "10.10.5.20", "nodes": ["server"]}  #can reach 10.10.3.0 (consumer2) subnet through router2 gw
]

print(f"Setting up slice {slice_name}")
print(f"Using sites {site1}, {site2}, {site3}, {site4}, {site5}")

## Check resources

Check if you have a pre-existing slice by this name already running

Don't attempt to submit the slice if you've already defined one by this name

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

## Node Set-up

Following example code, this will set-up the L2 & L3 networks per the configured lists

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=n['site'], 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell iterates each entry in the net_conf table
for n in net_conf:
    ifaces = []
    for node in n['nodes']:
        ifaces.append( slice.get_node(node["name"]).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] )
    slice.add_l2network(name=n["name"], interfaces=ifaces)

## Set-up the MFLib monitoring node
Create a node that will monitor the status of the other nodes

This step GREATLY increases slice requisition time. If debugging node links, do not add the MFLib monitoring tool.

In [ ]:
# Add measurement node to topology using static method.
MFLib.addMeasNode(slice, disk=100, image='docker_ubuntu_20')
print("MFLib Node added")

## Submit the slice for use

In [ ]:
slice.submit()

This step will pend until the above slice is ready for use. Helps prevent trying to configure nodes that don't exist.

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

## Configure Resources

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    print("")
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_config:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

## Ping Test
This step will ping each node from the consumers & server nodes. This step serves as a means to ensure all the connections are linked correclty and noone is "blind" to the netowrk.

In [ ]:
#Consumer 1
node = slice.get_node(name="consumer1")
try:
    stdout, stderr = node.execute(f'ping -c 1 consumer1')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 consumer2')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 router1')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 router2')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 server')
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
#Consumer 2
node = slice.get_node(name="consumer2")
try:
    stdout, stderr = node.execute(f'ping -c 1 consumer1')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 consumer2')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 router1')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 router2')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 server')
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
#Server
node = slice.get_node(name="server")
try:
    stdout, stderr = node.execute(f'ping -c 1 consumer1')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 consumer2')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 router1')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 router2')
except Exception as e:
    print(f"Exception: {e}")
print("====")
try:
    stdout, stderr = node.execute(f'ping -c 1 server')
except Exception as e:
    print(f"Exception: {e}")

## Draw the Topology

Draw the configuration based on the list arguments declared towards the start of this Notebook.

Note -- This does not draw from what FABRIC actually created. It draws what we wanted FABRIC to create. If methods aren't called correctly there will be a disconnect between these two.

In [ ]:
l2_nets = []
hosts   = []
l3_nets = []
for n in slice.get_l2networks():
    print( "L2 " + n.get_name() )
    l2_nets.append( (n.get_name(), {'color': 'lavender'}) )
for n in slice.get_l3networks():
    print( "L3 " + n.get_name() )
    l3_nets.append( (n.get_name(), {'color': 'pink'}) )
for n in slice.get_nodes():
    print( "Node " + n.get_name() )
    hosts.append( (n.get_name(), {'color': 'lightblue'}) )
nodes   = l2_nets + l3_nets + hosts

edges = []
for iface in slice.get_interfaces():
    ifDict = iface.toDict()
    edges.append( (ifDict['network'], ifDict['node'], {'label': ifDict['physical_dev'] + '\n' + ifDict['ip_addr'] + '\n' + ifDict['mac']}) )
     

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);

nx.draw_networkx_edge_labels(G, pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

## Log into nodes
Get the ssh commands for the Consumers, Routers, and Server

In [ ]:
print( slice.get_node("consumer1").get_ssh_command() )

In [ ]:
print( slice.get_node("consumer2").get_ssh_command() )

In [ ]:
print( slice.get_node("server").get_ssh_command() )

In [ ]:
print( slice.get_node("router1").get_ssh_command() )

In [ ]:
print( slice.get_node("router2").get_ssh_command() )

In [ ]:
print( slice.get_node("meas-node").get_ssh_command() )

## Initialize MFLib object

In [ ]:
%%time
mf = MFLib(slice_name)

## Install Prometheus and Grafana

Install the 2 tools and then get the access information

In [ ]:
%%time
instrumetize_results = mf.instrumentize( ["prometheus"] )

In [ ]:
# Grafana SSH Tunnel Command
# mf.grafana_tunnel_local_port = 10010 # optionally change the port
print(mf.grafana_tunnel)

print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

In [ ]:
# The grafana_manager service was created by the mf.instrumentize call.
# Get access info for Grafana by using the mflib.info call to the grafana_manager.
# Create a dictionary to pass to the service.
data = {}
# Set the info you want to get.
data["get"] = ["admin_password"]
# Call info using service name and data dictionary.
info_results = mf.info("grafana_manager", data)
print(info_results)

## Formulate Prometheus Queries
The following steps help establish a means to query the Prometheus data set from the Command Line

In [ ]:
from mflib.data_transfer import PrometheusExporter
prom_tools = PrometheusExporter(slice_name=slice_name)

In [ ]:
# Get prometheus admin credentials so we can create snapshots
data={}
data["get"] = ["ht_user", "ht_password"]
prom_credentials = prom_tools.info("prometheus", data)
print( prom_credentials )

Using the above logins to replace the `--user "username":"password"` in the following command
The query still needs refining as it returns a timestamp at the moment

curl -G -k --user 'xypifhBC':'dElMwDyq' --data-urlencode 'query=node_network_receive_packets_total{instance="router2",job="node", device="enp9s0"}' https://localhost:9090/api/v1/query

## Set up video stream components (IN WORK)

### Server Config

SSH into Router 1 from a terminal and then execute the following

` sudo apt update ; sudo apt install -y apache2 ; wget https://nyu.box.com/shared/static/d6btpwf5lqmkqh53b52ynhmfthh2qtby.tgz -O media.tgz ; sudo tar -v -xzf media.tgz -C /var/www/html `

### Router 1 Config

SSH into Router 1 from a terminal and then execute the following

`git clone https://github.com/NYU-METS/Main nyc-traces ; sudo apt update ; sudo apt install -y unrar-free ; unrar nyc-traces/Dataset/Dataset_1.rar ; wget https://raw.githubusercontent.com/teaching-on-testbeds/adaptive-video/main/rate-vary.sh -O ~/rate-vary.sh ; wget https://raw.githubusercontent.com/teaching-on-testbeds/adaptive-video/main/rate-set.sh -O ~/rate-set.sh `

Before you can execute the "rate-set.sh", you will need to go in and edit one of the lines. Open "rate-set.sh" in vi

Change the first line from

`addr=$(dig +short @127.0.0.53 romeo)`

to

`addr=$(dig +short @127.0.0.53 consumer1)`

Then run (The first time you run it, you may see an error referencing a problem deleting a qdisc, but you can safely ignore this error.)

` bash rate-set.sh 300Kbit `

### Router 2 Config

SSH into Router 2 from a terminal and then execute the following

`git clone https://github.com/NYU-METS/Main nyc-traces ; sudo apt update ; sudo apt install -y unrar-free ; unrar nyc-traces/Dataset/Dataset_1.rar ; wget https://raw.githubusercontent.com/teaching-on-testbeds/adaptive-video/main/rate-vary.sh -O ~/rate-vary.sh ; wget https://raw.githubusercontent.com/teaching-on-testbeds/adaptive-video/main/rate-set.sh -O ~/rate-set.sh `

Before you can execute the "rate-set.sh", you will need to go in and edit one of the lines. Open "rate-set.sh" in vi

Change the first line from

`addr=$(dig +short @127.0.0.53 romeo)`

to

`addr=$(dig +short @127.0.0.53 consumer1)`

You will then, in the same file, need to duplicate everything so that you can do the same steps for "consumer2" in a single script call

Then run (The first time you run it, you may see an error referencing a problem deleting a qdisc, but you can safely ignore this error.)

` bash rate-set.sh 400Kbit `

### Consumer 1 Config

SSH into Consumer 1 from a terminal and then execute the following

` git clone https://github.com/teaching-on-testbeds/AStream ; sudo apt update ; sudo apt install -y python3 ffmpeg ` 

### Consumer 2 Config

SSH into Consumer 2 from a terminal and then execute the following

` git clone https://github.com/teaching-on-testbeds/AStream ; sudo apt update ; sudo apt install -y python3 ffmpeg ` 

## Running Video Stream

Once Configured, in the Consumer 1 & 2 terminals you will execute

` python3 ~/AStream/dist/client/dash_client.py -m http://server/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d `

## Preparing Data for Collection

At the end of the experiment you will need to prep the video stream for either or both consumers using

` cp $(ls -t1  ~/ASTREAM_LOGS/DASH_BUFFER_LOG_*  | head -n 1 ) ~/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv ; suffix=$(ls -lt | grep "TEMP_" | head -n 1 | cut -f2 -d"_") ; cd ~/TEMP_$suffix ; rm -f ~/BigBuckBunny.mp4 ; cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4 ; ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy ~/BigBuckBunny.mp4 ; cd ~ `

## DEBUG Route Switching

In [ ]:
slice.get_node(name="consumer1").ip_route_del(subnet=IPv4Network("10.10.5.0/24"), gateway="10.10.2.20")

In [ ]:
slice.get_node(name="consumer1").ip_route_del(subnet=IPv4Network("10.10.4.0/24"), gateway="10.10.1.10")

In [ ]:
slice.get_node(name="consumer1").ip_route_add(subnet=IPv4Network("10.10.5.0/24"), gateway="10.10.2.20")

In [ ]:
slice.get_node(name="consumer1").ip_route_add(subnet=IPv4Network("10.10.4.0/24"), gateway="10.10.1.10")

In [ ]:
ifaceR1 = slice.get_node("consumer1").get_component("net_c1r1").get_interfaces()[0] #10.10.4.0 subnet path
ifaceR2 = slice.get_node("consumer1").get_component("net_c1r2").get_interfaces()[0] #10.10.5.0 subnet path
#ifaceR1.ip_link_down()
#ifaceR1.ip_link_up()

In [ ]:
tmpIface = slice.get_node("router2").get_component("net_c1r2").get_interfaces()[0] #10.10.4.0 subnet path
print( tmpIface.get_bandwidth() )

## Cleanup
Step to delete the slice

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.delete()